In [1]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
import shutil

In [2]:
def get_image_creation_date(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if exif_data is not None:
            for tag, value in exif_data.items():
                if TAGS.get(tag) == 'DateTimeOriginal':
                    return value.split(" ")[0] # nur das Datum zurückgeben
    except (IOError, AttributeError):
        pass

    return None



In [3]:
def get_list_of_creation_date(image_folder):
    images= []
    
    for filename in os.listdir(image_folder):
        if filename != ".DS_Store":
            image_path = os.path.join(image_folder, filename)
            with Image.open(image_path) as img:
                try:
                    creation_date = img._getexif()[36867]
                except:
                    print('no creation date')
                    creation_date= "2023:07:30 10:00:00"
                images.append((image_path, creation_date))
    return images

In [9]:
def rename_and_resize_files(images, sorted_images, out_path, max_dim, team):
    counter = 0
    for index, (image_path,creation_date) in enumerate(sorted_images):
        
        date = creation_date.split(" ")[0]
        date = date.replace(':','')
        new_filename = f"{date}_{team}_{str(index).zfill(3)}.jpg"
        new_filepath = os.path.join(out_path, new_filename)
        
        with Image.open(image_path) as img:
            img.thumbnail((max_dim, max_dim), Image.ANTIALIAS)
            img.save(new_filepath)
            print(f"Renamed and resized: {new_filepath}")

In [11]:
directory = '/Users/sandra/Downloads/Kamera/'
images = get_list_of_creation_date(directory)

sorted_images = sorted(images, key=lambda x: x[1])

output_folder = '/Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/'
rename_and_resize_files(images, sorted_images, output_folder, 1200, 'C-Junioren_Turnier_Steinheim')
#rename_and_resize_files(directory, out_path, 1200, 'A-Junioren_Kobra_Abschluss')

/Users/sandra/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  # This is added back by InteractiveShellApp.init_path()


Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_000.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_001.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_002.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_003.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_004.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_005.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_006.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/ABI-Homepage/images/2023/09/20230908_C-Junioren_Turnier_Steinheim_

In [8]:
def read_text_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

In [32]:
def find_paragraphs(text):
    paragraphs = text.split('\n\n')
    return paragraphs

In [39]:
file_path = '/Users/sandra/Downloads/Kamera/text.rtf'
text_content = read_text_file(file_path)
paragraphs = find_paragraphs(text_content)

print(f"Anzahl der Absätze: {len(paragraphs)}")
for i, paragraph in enumerate(paragraphs, 1):
    print(f"Absatz {i}:<p>")
    print(paragraph)
    print("</p>-----------------------")

Anzahl der Absätze: 5
Absatz 1:<p>
Sommerfußballcamp
</p>-----------------------
Absatz 2:<p>
In der Zeit vom 27.07. bis 30.07.2023 fand das zweite Fußballcamp der Fußballschule Frieböse-Hofmeier in Beilstein statt. Die 35 fußballbegeisterten Kinder und Jugendliche trotzten dem nasskalten Wetter und konnten wieder ihre Fähigkeiten an den vier sehr intensive Trainingstage weiter entwickeln.
</p>-----------------------
Absatz 3:<p>
Das Camp bot vielfältige Trainingseinheiten mit Schwerpunkten wie Koordination, Ballbehandlung und Torabschluss sowie zahlreiche Spielformen, die das Zusammenspiel und taktische Spielverständnis förderten. Die Fußballschule von Matthias Frieböse und Christian Hofmeier legt besonderen Wert darauf, dass die Kinder während des Camps Fortschritte erzielen aber dabei der Spaß am Fußball nicht zu kurz kommt.
</p>-----------------------
Absatz 4:<p>
Am Ende jedes Fußballcamps erhalten die jungen Teilnehmer und deren Eltern eine Rückmeldung über das Gelernte und Anreg

In [38]:
import gdown
save_dir = "/Users/sandra/Documents/Test_Download/"
url= f"https://drive.google.com/uc?id={avysr5uUrKsWrUjb7}&export=download"
gdown.download(url, output=save_dir, quiet=False)

NameError: name 'avysr5uUrKsWrUjb7' is not defined

In [9]:
folder_id ="https://photos.google.com/u/1/album/AF1QipNLWawxiQbUlceavLDhDCR6FulXkbraZBRTlyNS"
save_dir = "/Users/sandra/Documents/Test_Download/"
download_photos_from_google_folder(folder_id, save_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'